In [1]:
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
dir_train = 'E:/Jupyter NB/10 class Classification/Images'

image_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test'

train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

In [3]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 33, 33, 128)      

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

Found 1233 images belonging to 10 classes.
Found 200 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


# Model Compilation

In [5]:
from keras.losses import categorical_crossentropy as logloss
from keras import backend as K

def loss(y_true, y_pred):
    entropy = -K.mean(K.sum(y_pred*K.log(y_pred), 1))
    beta = 0.1
    return logloss(y_true, y_pred) - beta*entropy

In [6]:
from keras import optimizers

model.compile(loss=loss,
    optimizer=optimizers.SGD(lr=1e-2),
    metrics=['categorical_crossentropy', 'accuracy'])

# Training

In [7]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 49s 491ms/step - loss: 1.9766 - categorical_crossentropy: 2.2008 - accuracy: 0.2403 - val_loss: 2.0199 - val_categorical_crossentropy: 2.2223 - val_accuracy: 0.2350
Epoch 2/10
100/100 [==============================] - 43s 426ms/step - loss: 1.7385 - categorical_crossentropy: 1.9444 - accuracy: 0.3441 - val_loss: 1.4286 - val_categorical_crossentropy: 1.8663 - val_accuracy: 0.3400
Epoch 3/10
100/100 [==============================] - 35s 346ms/step - loss: 1.5356 - categorical_crossentropy: 1.7219 - accuracy: 0.4451 - val_loss: 2.0160 - val_categorical_crossentropy: 1.9411 - val_accuracy: 0.3450
Epoch 4/10
100/100 [==============================] - 34s 341ms/step - loss: 1.3874 - categorical_crossentropy: 1.5589 - accuracy: 0.4937 - val_loss: 1.3800 - val_categorical_crossentropy: 1.6186 - val_accuracy: 0.4650
Epoch 5/10
100/100 [==============================] - 34s 344ms/step - loss: 1.2684 - categorical_crossentropy: 1.4285 - acc

In [8]:
model.evaluate_generator(test_generator, 150)

[1.4539083242416382, 1.851881504058838, 0.4950000047683716]

In [9]:
model.save_weights('teacher_weights.hdf5')

In [10]:
'''
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
'''

10/10 [==============================] - 2s 197ms/step


In [11]:
'''
STEP_SIZE_TEST=train_generator.n//train_generator.batch_size
train_generator.reset()
pred=model.predict_generator(train_generator,
    steps=STEP_SIZE_TEST,
    verbose=1)
'''

61/61 [==============================] - 6s 100ms/step


In [12]:
from keras.models import Model

# remove softmax
model.layers.pop()
model = Model(model.input, model.layers[-1].output)
# now model outputs logits

In [22]:
from tqdm import tqdm
import numpy as np

batches = 0
train_logits = {}

for x_batch, name_batch in tqdm(train_generator):
    print(np.shape(x_batch))
    #print(name_batch)
    #print(len(x_batch))
    #print(x_batch)
    
    batch_logits = model.predict_on_batch(x_batch)
    print(np.shape(batch_logits))
    #print(batch_logits)
    for i, n in enumerate(name_batch):
        print(batch_logits[i])
        print(n)
    
    for i, n in enumerate(name_batch):
        #print(i)
        #print(n)
        #print(train_logits)
        #print(batch_logits[i])
        train_logits[n] = batch_logits[i]
    '''
    batches += 1
    if batches >= 400: # 25600/64
        break
    '''

  0%|                                                                                           | 0/62 [00:00<?, ?it/s]

(20, 299, 299, 3)
(20, 10)
[3.70864844e-04 3.08667404e-06 9.89612639e-01 5.03740180e-03
 2.01689458e-04 9.50726331e-04 1.06929794e-04 1.39943499e-03
 2.21049669e-03 1.06748725e-04]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[1.2886509e-02 1.0360724e-02 6.8904728e-02 1.8360894e-02 5.8740536e-03
 1.2576629e-02 7.5834996e-01 1.8690531e-04 1.9583262e-03 1.1054129e-01]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[3.4284876e-06 1.4483948e-03 9.8794252e-01 7.2967913e-03 2.3657728e-04
 8.4779695e-06 2.2480544e-04 1.2964559e-03 1.3486993e-03 1.9384523e-04]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[6.4133684e-04 3.4523496e-04 4.2557633e-01 5.7096785e-01 1.2763745e-04
 7.5503206e-04 5.5961701e-04 3.9172155e-04 4.0751538e-04 2.2770009e-04]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.79164964 0.00844734 0.00947156 0.02556248 0.03335799 0.03047122
 0.01003691 0.02287291 0.036985   0.03114498]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[2.6692247e-03 7.4775526e-05 3.8005976e-04 3.9239801e-03 9.8496282e-01
 3.4750028e-06 7.4976543e-03 4.4595698e-04 3.8

TypeError: unhashable type: 'numpy.ndarray'